# Smart insole activity modelling
 
In this notebook we analyse some time series data taken from the sensors on the insole and attempt to fit a predictive model to determine what kind of activity is being performed.

## Overview
### The data

The [../data](data) consists of timestamped sensor readings from the device. Timestamps are in ISO-8601 datetime format and sensor readings are integer values representing pressure (force) in kg.

Example: `2019-11-18T10:00:00.001,1,15`

Which is 15 kg from sensor number 1 at 2019-11-18T10:00:00.001.

### Analysis

We first want to develop some general techniques for locating trends, correlations and periodicity in the sensor data, and this will lead us to make appropriate modelling choices. We also want to uncover statistical distributions in the data which will enable us to generate fake data that looks similar; fake data is useful for integration testing.

In [ ]:
# Preamble
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(11, 4)})

# Data loading

In [ ]:
pressure_data = pd.read_csv("../data/sample.csv", index_col=0, parse_dates=[0])

In [ ]:
pressure_data.query('date != "NaT"')

# Data exploration

In [ ]:
pressure_data.head()

In [ ]:
pressure_data.dtypes

In [ ]:
pressure_data['pressure'].plot(linewidth=0.5)